In [ ]:
import torch
import numpy as np
import os

os.chdir('/content/drive/MyDrive/dagaare_words_definition/')
dagaare_Tokenizer= "/content/Transformer.py"
english_file = "/content/drive/MyDrive/dagaare_words_definition/dagaare_definition_sentences.txt"
dagaare_file = "/content/drive/MyDrive/dagaare_words_definition/dagaare_words_sentences.txt"

In [ ]:
from Transformer import Transformer

In [ ]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

dagaare_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                      'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                        'Y', 'Z',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z','à','á','â','ã','æ','è','é','ì','í','ò','ó','õ','ù','ú','û','ĩ','ŋ','ŏ','ũ',
                        'Ɔ','Ɛ','ǎ','ɔ','ɛ','ɡ','ɪ','ʊ','ʋ','ͻ','ε','ṹ','ạ','ẽ','ὸ','ό',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                        'Y', 'Z',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [ ]:
index_to_dagaare = {k:v for k,v in enumerate(dagaare_vocabulary)}
dagaare_to_index = {v:k for k,v in enumerate(dagaare_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [ ]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(dagaare_file, 'r') as file:
    dagaare_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
dagaare_sentences = dagaare_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
dagaare_sentences = [sentence.rstrip('\n') for sentence in dagaare_sentences]

In [ ]:
english_sentences[:10]

['definition',
 '(1) and',
 '(2) in order to',
 'but',
 'they',
 'the',
 'to hate',
 'expression of surprise and regret',
 'expression of regret',
 'expression expressing “i told you so.”']

In [ ]:
dagaare_sentences[:10]

['Word', 'a', 'a', 'a', 'a', 'a', 'a', 'aa', 'aa', 'aa']

In [ ]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length dagaare: {np.percentile([len(x) for x in dagaare_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length dagaare: 21.0
97th percentile length English: 75.0


In [ ]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(dagaare_sentences)):
    dagaare_sentence, english_sentence = dagaare_sentences[index], english_sentences[index]
    if is_valid_length(dagaare_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(dagaare_sentence, dagaare_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(dagaare_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 8112
Number of valid sentences: 7209


In [ ]:
dagaare_sentences = [dagaare_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [ ]:
import string

# Function to remove punctuation marks and special characters from text
def remove_special_characters(text):
    special_chars = ['“','́','…','\t','–', '”','•', '’','‘','æ','ɪ','ạ', 'à', 'á', 'â', 'ã', 'è', 'é', 'ì', 'í', 'ò', 'ó', 'õ', 'ù', 'ú', 'û', 'ĩ', 'ŋ', 'ŏ', 'ũ', 'Ɔ', 'Ɛ', 'ǎ', 'ɔ', 'ɛ', 'ɡ', 'ʊ', 'ʋ', 'ͻ', 'ε', 'ṹ', 'ẽ', 'ὸ', 'ό']
    translator = str.maketrans('', '', string.punctuation)
    cleaned_text = text.translate(translator)
    for char in special_chars:
        cleaned_text = cleaned_text.replace(char, 'UNK')
    return cleaned_text

# Paths to the English and Dagaare files
english_file = "/content/drive/MyDrive/dagaare_words_definition/dagaare_definition_sentences.txt"
dagaare_file = "/content/drive/MyDrive/dagaare_words_definition/dagaare_words_sentences.txt"

# Read the contents of the English and Dagaare files
with open(english_file, 'r', encoding='utf-8') as f:
    english_sentences = f.readlines()

with open(dagaare_file, 'r', encoding='utf-8') as f:
    dagaare_sentences = f.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 500000
english_sentences = english_sentences[:TOTAL_SENTENCES]
dagaare_sentences = dagaare_sentences[:TOTAL_SENTENCES]

# Remove special characters and strip newline characters from English and Dagaare sentences
english_sentences = [remove_special_characters(sentence.rstrip('\n')) for sentence in english_sentences]
dagaare_sentences = [sentence.rstrip('\n') for sentence in dagaare_sentences]

In [ ]:
dagaare_sentences[:3]

['Word', 'a', 'a']

In [ ]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 1000
dg_vocab_size = len(dagaare_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          dg_vocab_size,
                          english_to_index,
                          dagaare_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [ ]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(97, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, dagaare_sentences):
        self.english_sentences = english_sentences
        self.dagaare_sentences = dagaare_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.dagaare_sentences[idx]

In [ ]:
dataset = TextDataset(english_sentences, dagaare_sentences)

In [ ]:
len(dataset)

8112

In [ ]:
dataset[1]

('1 and', 'a')

In [ ]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('Definition', '1 and', '2 in order to', 'but', 'they', 'the', 'to hate', 'expression of surprise and regret', 'expression of regret', 'expression expressing UNKI told you soUNK', 'expression of pain', '1 expression of disapproval', '1 yes answer to a call', 'is that so expression of uncertain surprise', 'answer in agreement', 'expression of lack of confidence in someoneUNKs ability to perform a task', 'expression of total despair', 'infertile woman ', 'well then', '1 expression of despair over a repeated mishap', '1 exclamation of satisfaction or commendation', '1 palm tree 2 palm fruit', 'palm fruit', 'palm wine', 'palm oil', 'palmfrond broom', 'palm tree', 'which ones nonhuman', 'Africa', 'African'), ('Word', 'a', 'a', 'a', 'a', 'a', 'a', 'aa', 'aa', 'aa', 'aae', 'a-a', 'ãa', 'ãaa', 'âaa', 'aa-ee', 'aa-hii', 'aane', 'aaŋ', 'aba', 'abaa', 'abɛ', 'abɛbiri', 'abɛdãã', 'abɛkãã', 'abɛsaare', 'abɛteɛ', 'abuobo', 'Afereka', 'Aferekaneɛ')]
[('preface of a book', '1 to dance 2 to jump up an

In [ ]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=dagaare_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
NEG_INFTY = -1e9

def create_masks(eng_batch, dg_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, dg_sentence_length = len(eng_batch[idx]), len(dg_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      dg_chars_to_padding_mask = np.arange(dg_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, dg_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, dg_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, dg_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, dg_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, dg_batch)
        optim.zero_grad()
        dg_predictions = transformer(eng_batch,
                                     dg_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(dg_batch, start_token=False, end_token=True)
        loss = criterian(
            dg_predictions.view(-1, dg_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == dagaare_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"dagaare Translation: {dg_batch[0]}")
            dg_sentence_predicted = torch.argmax(dg_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in dg_sentence_predicted:
              if idx == dagaare_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_dagaare[idx.item()]
            print(f"dagaare Prediction: {predicted_sentence}")


            transformer.eval()
            dg_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, dg_sentence)
                predictions = transformer(eng_sentence,
                                          dg_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_dagaare[next_token_index]
                dg_sentence = (dg_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {dg_sentence}")
            print("-------------------------------------------")

Epoch 0
Contents of x: ('Definition', '1 and', '2 in order to', 'but', 'they', 'the', 'to hate', 'expression of surprise and regret', 'expression of regret', 'expression expressing UNKI told you soUNK', 'expression of pain', '1 expression of disapproval', '1 yes answer to a call', 'is that so expression of uncertain surprise', 'answer in agreement', 'expression of lack of confidence in someoneUNKs ability to perform a task', 'expression of total despair', 'infertile woman ', 'well then', '1 expression of despair over a repeated mishap', '1 exclamation of satisfaction or commendation', '1 palm tree 2 palm fruit', 'palm fruit', 'palm wine', 'palm oil', 'palmfrond broom', 'palm tree', 'which ones nonhuman', 'Africa', 'African')
Contents of y: ('Word', 'a', 'a', 'a', 'a', 'a', 'a', 'aa', 'aa', 'aa', 'aae', 'a-a', 'ãa', 'ãaa', 'âaa', 'aa-ee', 'aa-hii', 'aane', 'aaŋ', 'aba', 'abaa', 'abɛ', 'abɛbiri', 'abɛdãã', 'abɛkãã', 'abɛsaare', 'abɛteɛ', 'abuobo', 'Afereka', 'Aferekaneɛ')
Type of x: <cla

KeyError: '̃'

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  dg_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, dg_sentence)
    predictions = transformer(eng_sentence,
                              dg_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_dagaare[next_token_index]
    dg_sentence = (dg_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return dg_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)